In [1]:
import numpy as np
import reedsolo

def encode_RS_rows(matrix: np.ndarray, nsym: int):
    rs = reedsolo.RSCodec(nsym)
    encoded = []
    for row in matrix:
        bytes_row = row.astype(np.float16).tobytes()
        encoded_bytes = rs.encode(bytes_row)
        encoded.append(np.frombuffer(encoded_bytes, dtype=np.uint8))
    return np.array(encoded)

def decode_RS_rows(matrix_bytes: np.ndarray, nsym: int, original_len: int):
    rs = reedsolo.RSCodec(nsym)
    decoded = []
    for row in matrix_bytes:
        try:
            decoded_bytes = rs.decode(row.tobytes())[0]
            decoded.append(np.frombuffer(decoded_bytes, dtype=np.float16))
        except reedsolo.ReedSolomonError:
            decoded.append(np.full(original_len, np.nan))  # ошибка — заполним NaN
    return np.stack(decoded)

In [2]:
def flip_random_bit(arr, k):
    arr_copy = arr.copy()
    arr_uint32 = arr_copy.view(np.int8)
    if arr_uint32.size == 0:
        return arr_copy  # Пустой массив
    random_index = np.random.randint(0, arr_uint32.size)
    number = arr_uint32.flat[random_index]
    arr_uint32.flat[random_index] ^= np.int8(1 << k)
    print("[DECODED] source number: {}, flipped number: {}".format(number, arr_uint32.flat[random_index]))
    return arr_copy

def float_flip_random_bit(arr, k):
    arr_copy = arr.copy()
    arr_uint32 = arr_copy.view(np.int16)
    if arr_uint32.size == 0:
        return arr_copy  # Пустой массив
    random_index = np.random.randint(0, arr_uint32.size)
    number = arr_uint32.flat[random_index]
    arr_uint32.flat[random_index] ^= np.uint16(1 << k)
    print("[NO DECODED] source number: {}, flipped number: {}".format(number, arr_uint32.flat[random_index]))
    return arr_copy

In [24]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

def plot_hist(arr, name, min, max):
    plt.figure(figsize=(10, 5))
    plt.hist(arr.flatten(), bins=range(min, max), edgecolor='black')
    plt.title(name)
    plt.xlabel("Значение")
    plt.ylabel("Частота")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

def pca_method(array, rate, info):
    pca = PCA()
    pca.fit(array)
    cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
    k = np.argmax(cumulative_variance >= rate) + 1
    print("[{}] for rate {} k = {}".format(info, rate, k))
    C = pca.components_[:k]
    mu = pca.mean_
    Z = (array - mu) @ C.T
    return Z, C, mu

def regular_matmul(X, W, bit):
    # bit = 13
    Y_correct = X @ W
    X_raw_corrupted = X.copy()
    X_raw_corrupted = float_flip_random_bit(X_raw_corrupted, 13)
    Y_broken = X_raw_corrupted @ W
    print("🔴 Без защиты:", np.linalg.norm(Y_broken - Y_correct))

def rs_matmul(X, W, bit, nsym, info):
    # bit = 7
    # plot_hist(X, "Source X", min=0, max=20)
    X_encoded = encode_RS_rows(X, nsym=nsym)
    print("[INFO] array: {}".format(X_encoded))
    X_encoded, C, mu = pca_method(X_encoded, 0.99, "PCA ENCODED")
    # _, _ = pca_method(X, 0.99, "PCA No ENCODE")
    # plot_hist(X_encoded, "Encoded X", min=0, max=300)
    Y_correct = X @ W
    X_encoded_corrupted = X_encoded.copy()
    X_encoded_corrupted = flip_random_bit(X_encoded_corrupted, bit)
    X_encoded_corrupted = (X_encoded_corrupted @ C + mu)
    X_encoded_corrupted = np.clip(np.round(X_encoded_corrupted), -128, 127).astype(np.int8)
    print("[INFO] X_encoded_corrupted.shape: {}".format(X_encoded_corrupted.shape))
    print("[INFO] array restored: {}".format(X_encoded_corrupted))
    X_restored = decode_RS_rows(X_encoded_corrupted, nsym=nsym, original_len=X.shape[1])
    Y_rs = X_restored @ W
    print("🟢 С RS-восстановлением:", np.linalg.norm(Y_rs - Y_correct))
    if info:
        print("type: {}, size: {}, bytes: {}".format(X.dtype, X.size, 16 * X.size))
        print("type: {}, size: {}, bytes: {}".format(X_encoded.dtype, X_encoded.size, 8 * X_encoded.size))
        print("type: {}, size: {}, bytes: {}".format(X_restored.dtype, X_restored.size, 16 * X_restored.size))
        print("rate of bytes: {}".format((8 * X_encoded.size) / (16 * X.size)))


In [25]:
import time

nsym = 2
n = 256
m = 256

# X = np.random.randn(n, m).astype(np.float16)
# W = np.random.randn(m, n).astype(np.float16)

X = np.random.randn(4, 5).astype(np.float16)
W = np.random.randn(5, 4).astype(np.float16)

start = time.time()
regular_matmul(X, W, 13)
total_regular = time.time() - start
print("-----------------[regular_matmul] Time: {} -----------------", total_regular)

start = time.time()
rs_matmul(X, W, 6, nsym, True)
total_rs = time.time() - start
print("-----------------[rs_matmul] Time: {} -----------------", total_rs)
print("Degradation: {}".format(total_rs / total_regular))

[NO DECODED] source number: 13999, flipped number: 5807
🔴 Без защиты: 0.929
-----------------[regular_matmul] Time: {} ----------------- 0.0002613067626953125
[INFO] array: [[104  43 235  57 159 184 163 188  52  56 131  38]
 [197 188 175  54  42 175 140 168  45  57  93   8]
 [224 185 244  53  48  55  45  60  55  60 243 118]
 [109 184 206  57  40  52 180 189  94 174 144  87]]
[PCA ENCODED] for rate 0.99 k = 3
[DECODED] source number: 87, flipped number: 23
[INFO] X_encoded_corrupted.shape: (4, 12)
[INFO] array restored: [[104  43 127  57 127 127 127 127  52  56 127  38]
 [127 127 127  54  42 127 127 127  45  57  93   8]
 [127 127 127  53  48  55  45  60  55  60 127 118]
 [109 127 127  57  40  52 127 127  94 127 127  87]]
🟢 С RS-восстановлением: nan
type: float16, size: 20, bytes: 320
type: float64, size: 12, bytes: 96
type: float64, size: 20, bytes: 320
rate of bytes: 0.3
-----------------[rs_matmul] Time: {} ----------------- 0.002053976058959961
Degradation: 7.860401459854015


In [ ]:
# def pca_encode(self, weights_uint8, rate):
#     weights_float = weights_uint8.astype(np.float32)
#     weights_pca_k = self.pca.fit_transform(weights_float)
#     explained_var = np.cumsum(self.pca.explained_variance_ratio_)
#     k = np.argmax(explained_var >= rate) + 1
#     print("explained var: {}".format(explained_var))
#     print("Compression: source dim = {}, compressed dim = {}".format(len(self.pca.explained_variance_ratio_), k))
#     weights_pca_k = weights_pca_k#[:, :k]
#     components_k = self.pca.components_#[:k]
#     mean = self.pca.mean_
#     return weights_pca_k, components_k, mean

# def bitflip(self, k: int):
#     # k in [0,...,7]
#     rand_ind = np.random.randint(0, self.weights_uint8_pca.ravel().size)
#     source_number = self.weights_uint8_pca.ravel()[rand_ind]
#     flipped = source_number ^ (1 << k)
#     corr_number = np.frombuffer(flipped.tobytes(), dtype=np.uint8)[0]
#     self.weights_uint8_pca.ravel()[rand_ind] = corr_number
#     print("source number: {}, corr number: {}".format(source_number, corr_number))

# def forward(self, X):
#     start = time.time()
#     weights_uint = self.decode_RS_rows(self.weights_pca_k)
#     restored = weights_uint @ self.components_k + self.mean
#     res = X @ restored
#     self.time_.append(time.time() - start)
#     return res